In [3]:
# MatchJointerRaw test
import torch

from starry.utils.config import Configuration
from starry.utils.model_factory import loadModel


config = Configuration.createOrLoad('configs/matchjointer1-test.yaml')
model = loadModel(config['model'])


c_time, c_pitch, c_velocity = torch.rand(1, 10) * 10000, torch.randint(128, (1, 10)), torch.rand(1, 10) * 127
s_time, s_pitch, s_velocity = torch.rand(1, 6) * 10000, torch.randint(128, (1, 6)), torch.rand(1, 6) * 127

matching, c_vec, s_vec = model(c_time, c_pitch, c_velocity, s_time, s_pitch, s_velocity)

print('matching:', matching.shape)
print('c_vec:', c_vec.shape)
print('s_vec:', s_vec.shape)


matching: torch.Size([1, 6, 10])
c_vec: torch.Size([1, 6, 128])
s_vec: torch.Size([1, 128, 10])


In [1]:
# dataset
import os
from starry.utils.config import Configuration
from starry.utils.dataset_factory import loadDataset


DATA_DIR = os.environ.get('DATA_DIR')

config = Configuration.create('configs/matchjointer1-test.yaml')
train, val = loadDataset(config, data_dir=DATA_DIR, device='cpu')

it = iter(val)
tensors = next(it)
print('tensors:', [(tensor.shape, tensor.dtype) for tensor in tensors])

next(it)
next(it)
next(it)
tensors = next(it)
print('cis:', tensors[-1])


/home/xunan/work/deep-starry-melody/env/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensors: [(torch.Size([2, 256]), torch.float32), (torch.Size([2, 256]), torch.int64), (torch.Size([2, 256]), torch.float32), (torch.Size([2, 256]), torch.float32), (torch.Size([2, 256]), torch.int64), (torch.Size([2, 256]), torch.float32), (torch.Size([2, 256]), torch.int64)]
cis: tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [3]:
# MatchJointer1Loss
import torch

from starry.utils.config import Configuration
from starry.utils.model_factory import loadModel


config = Configuration.createOrLoad('configs/matchjointer1-test.yaml')
model = loadModel(config['model'], postfix='Loss')

criterion = tensors[0:3]
sample = tensors[3:6]
ci = tensors[-1]
loss, metric = model(criterion, sample, ci)

print('loss:', loss)
print('metric:', metric)


loss: tensor(0.2664, grad_fn=<BinaryCrossEntropyBackward0>)
metric: {'acc': 0.0}
